In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModel
from tqdm import tqdm
import numpy as np
import os
import json
import matplotlib.pyplot as plt

SFT_MODEL = "allenai/Llama-3.1-Tulu-3-8B-SFT"
RL_MODEL  = "allenai/Llama-3.1-Tulu-3-8B-DPO"

tokenizer = AutoTokenizer.from_pretrained(SFT_MODEL)

model_sft = AutoModelForCausalLM.from_pretrained(
    SFT_MODEL,
    torch_dtype=torch.bfloat16,
    device_map="cpu",
    cache_dir="/shared/storage-01/huggingface/models/"
)
model_rl = AutoModelForCausalLM.from_pretrained(
    RL_MODEL,
    torch_dtype=torch.bfloat16,
    device_map="cpu",
    cache_dir="/shared/storage-01/huggingface/models/"
)

all_deltas = []
param_sizes = []
sft_params = []
rl_state_dict = model_rl.state_dict()
sft_state_dict = model_sft.state_dict()

for name_sft, param_sft in tqdm(model_sft.named_parameters()):
    if name_sft not in rl_state_dict:
        print("woops")
for name_rl, param_rl in tqdm(model_rl.named_parameters()):
    if name_rl not in sft_state_dict:
        print("woops")

delta_dict = {}
with torch.no_grad():
    for name_sft, param_sft in tqdm(model_sft.named_parameters()):
        try:
            param_rl = rl_state_dict[name_sft].to(param_sft.device)
            delta = param_rl - param_sft.data
            delta_dict[name_sft] = delta
        except Exception as e:
            print(e)

def _to2d(x):
    if x.ndim == 2:
        return x
    return x.reshape(x.shape[0], -1)

def _sv(x2d):
    return torch.linalg.svdvals(x2d.float().cpu())

def _energy_rank(s, eps):
    if s.numel() == 0:
        return 0
    e = s**2
    cum = torch.cumsum(e, 0)
    total = cum[-1]
    if total <= 0:
        return 0
    target = (1.0 - eps) * total
    k = int(torch.searchsorted(cum, target).item()) + 1
    return min(k, s.numel())

def _effective_rank(s, tiny=1e-12):
    if s.numel() == 0:
        return 0.0
    p = s / (s.sum() + tiny)
    p = torch.clamp(p, min=tiny)
    H = -(p * torch.log(p)).sum()
    return float(torch.exp(H).item())

def _stable_rank(s, tiny=1e-12):
    if s.numel() == 0:
        return 0.0
    num = float((s**2).sum().item())
    den = float((s.max().item() ** 2) + tiny)
    return num / den

def _energy_curve(s):
    if s.numel() == 0:
        return None, None
    e = s**2
    cum = torch.cumsum(e, 0)
    frac = cum / (cum[-1] + 1e-12)
    k = torch.arange(1, s.numel() + 1)
    return k, frac

plot_dir = "rank_plots"
os.makedirs(plot_dir, exist_ok=True)

eps_list = [1e-1, 5e-2, 1e-2, 1e-3]

max_rank = []
rank = []

layer_metrics = {}

for key in tqdm(delta_dict.keys()):
    if 'layernorm' in key or len(delta_dict[key].shape) <= 1:
        continue

    A = _to2d(delta_dict[key])
    s = _sv(A)

    rank_ = int(torch.linalg.matrix_rank(A.float()).item())
    max_rank_ = min(A.shape)

    max_rank.append(max_rank_)
    rank.append(rank_)

    enr = {str(eps): _energy_rank(s, eps) for eps in eps_list}
    effr = _effective_rank(s)
    stabr = _stable_rank(s)

    layer_metrics[key] = {
        "shape_2d": list(A.shape),
        "max_rank": max_rank_,
        "exact_rank": rank_,
        "effective_rank": effr,
        "stable_rank": stabr,
        "energy_rank": enr
    }

    k, frac = _energy_curve(s)
    if k is not None:
        plt.figure()
        plt.plot(k.numpy(), frac.numpy())
        plt.xlabel("rank")
        plt.ylabel("normalized energy retained")
        plt.ylim(0, 1.01)
        plt.title(key)
        for eps in eps_list:
            ek = enr[str(eps)]
            plt.axvline(ek, linestyle="--", alpha=0.5)
        plt.tight_layout()
        plt.savefig(os.path.join(plot_dir, key.replace("/", "_") + ".png"), dpi=150)
        plt.close()

    print(layer_metrics[key])

mean = sum(y / x for x, y in zip(max_rank, rank)) / len(max_rank)
print(mean)

with open("layer_rank_metrics.json", "w") as f:
    json.dump(layer_metrics, f, indent=2)
print("plots ->", plot_dir)
print("metrics -> layer_rank_metrics.json")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/439 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/875 [00:00<?, ?B/s]

...

{'shape_2d': [14336, 4096], 'max_rank': 4096, 'exact_rank': 4096, 'effective_rank': 3767.5869140625, 'stable_rank': 20.986714950114134, 'energy_rank': {'0.1': 2807, '0.05': 3308, '0.01': 3884, '0.001': 4071}}


 99%|█████████▊| 287/291 [18:32<00:24,  6.08s/it]

{'shape_2d': [4096, 14336], 'max_rank': 4096, 'exact_rank': 4096, 'effective_rank': 3655.259033203125, 'stable_rank': 8.56636480204158, 'energy_rank': {'0.1': 2594, '0.05': 3161, '0.01': 3835, '0.001': 4065}}


100%|██████████| 291/291 [19:09<00:00,  3.95s/it]

{'shape_2d': [128264, 4096], 'max_rank': 4096, 'exact_rank': 4096, 'effective_rank': 4002.978759765625, 'stable_rank': 163.5574989758626, 'energy_rank': {'0.1': 3393, '0.05': 3719, '0.01': 4005, '0.001': 4080}}
0.9975780385785398
plots -> rank_plots
metrics -> layer_rank_metrics.json


In [2]:
import json, re, os
import numpy as np
import matplotlib.pyplot as plt

with open("layer_rank_metrics.json", "r") as f:
    layer_metrics = json.load(f)

def layer_idx(name):
    m = re.search(r"model\.layers\.(\d+)\.", name)
    return int(m.group(1)) if m else None

def layer_type(name):
    if ".self_attn.q_proj." in name: return "Q"
    if ".self_attn.k_proj." in name: return "K"
    if ".self_attn.v_proj." in name: return "V"
    if ".self_attn.o_proj." in name: return "O"
    if ".mlp.gate_proj." in name: return "GATE"
    if ".mlp.up_proj." in name: return "UP"
    if ".mlp.down_proj." in name: return "DOWN"
    return None

flat = {}
for name, d in layer_metrics.items():
    i = layer_idx(name)
    t = layer_type(name)
    if i is None or t is None:
        continue
    base = {
        "max_rank": d["max_rank"],
        "exact_rank": d["exact_rank"],
        "exact_rank_ratio": d["exact_rank"] / (d["max_rank"] + 1e-12),
        "effective_rank": d["effective_rank"],
        "stable_rank": d["stable_rank"],
    }
    enr = d.get("energy_rank", {})
    for eps_str, k in enr.items():
        base[f"energy_rank_eps={eps_str}"] = k
        base[f"energy_rank_ratio_eps={eps_str}"] = k / (d["max_rank"] + 1e-12)
    flat.setdefault(t, []).append((i, base))

types = sorted(flat.keys())
metrics_keys = sorted({k for t in types for _, b in flat[t] for k in b.keys()})

out_dir = "metric_vs_depth_plots"
os.makedirs(out_dir, exist_ok=True)

for mk in metrics_keys:
    plt.figure()
    for t in types:
        items = sorted(flat[t], key=lambda x: x[0])
        xs = [i for i, _ in items]
        ys = [b[mk] for _, b in items]
        plt.plot(xs, ys, label=t)
    plt.xlabel("depth (layer index)")
    plt.ylabel(mk)
    plt.title(mk + " vs depth")
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(out_dir, mk.replace("/", "_") + ".png"), dpi=150)
    plt.close()

print("saved ->", out_dir)


saved -> metric_vs_depth_plots
